In [1]:
import sftrident.qft as qft
import numpy as np
import Konstanten as con
import Grid_cms as grid

/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: n

In [2]:
m=1.0


def eps_laser(l):
    return (l==1)*qft.MinkowskiVector( [0,1,0,0] ) + (l==2)*qft.MinkowskiVector( [0,0,1,0] )

def eps_photon(l):
    return (l==1)*qft.MinkowskiVector([0,np.cos(theta_ph)*np.cos(phi_ph),np.cos(theta_ph)*np.sin(phi_ph),-np.sin(theta_ph)])\
           +(l==2)*qft.MinkowskiVector( [0,-np.sin(phi_ph),np.cos(phi_ph),0])


w_laser     = 0.75/m
w_photon    = w_laser
w_cms       = w_laser
w           = w_laser
sqrt_s      = 2.*w_laser

pplusp = w_laser

a0            = 1.0
sigma         = 25 #pulse length
w_laser_l     = 1.e0
w_photon_l    = 4.e0
ksi           = np.pi/4.

Genau_theta   = 5
Genau_gamma   = 5

Harmonic_max  = 1


Envelope      = 'cos^2'

m_eff         = m*np.sqrt(1+0.5*a0**2)


theta_ph      = 0.
phi_ph        = 0.
phi_pos       = 0.


Abstand   = 2.0

y0_min  = lambda l: np.log ( w_cms/m - np.sqrt( w_cms**2/m**2 - 1./l ) )
y1_max  = lambda l: np.log ( w_cms/m + np.sqrt( w_cms**2/m**2 - 1./l ) )

y_z    = qft.parray(np.linspace(y0_min(Harmonic_max + Abstand),y1_max(Harmonic_max + Abstand),Genau_theta),axis=1)[np.newaxis,:]
#p_perp=qft.parray(1.0)
#print "pT: %s"%(p_perp)
#m_perp  = sqrt(1 + p_perp**2)
p_perp  = qft.parray(grid.Grid_rap(y_z[0,:], w_cms, 0 , Harmonic_max + Abstand , Genau_gamma , a0)) #p_T
m_perp  = np.sqrt(1 + p_perp**2)


#y_z    = parray(linspace(y0_min(Harmonic_max + Abstand),y1_max(Harmonic_max + Abstand),Genau_theta),axis=1)[newaxis,:]
#y_z = qft.parray(np.log(pplusp/m_perp))

#print "yz: %s"%(y_z)


In [3]:

m=1.0
def kinematik():

    k_laser       = qft.MinkowskiVector([w_laser,0,0,-w_laser])
    n1            = qft.MinkowskiVector([1, np.sin(theta_ph)*np.cos(phi_ph), np.sin(theta_ph)*np.sin(phi_ph), np.cos(theta_ph)])
    k_photon      = w_photon*n1

    p_pos  = qft.MinkowskiVector([ m_perp*np.cosh(y_z),
                            p_perp*np.cos(phi_pos),
                            p_perp*np.sin(phi_pos),
                            m_perp*np.sinh(y_z)] )

    q_pos      = p_pos + (m**2*a0**2)/(4.*(k_laser*p_pos))*k_laser # eff Mom Pos


    p_el_plus  =     k_photon.plus_component()     -     p_pos.plus_component()
    p_el_1     =     k_photon._1()                 -     p_pos._1()
    p_el_2     =     k_photon._2()                 -     p_pos._2()

    p_el_minus = (m**2+p_el_1**2+p_el_2**2)/p_el_plus

    p_el_0     = 0.5*(p_el_plus + p_el_minus)
    p_el_3     = 0.5*(p_el_plus - p_el_minus)
    p_el       = qft.MinkowskiVector([ p_el_0, p_el_1 ,p_el_2, p_el_3 ])


    eps_m      = eps_laser(1)*np.cos(ksi)-1j*eps_laser(2)*np.sin(ksi)
    eps_p      = eps_laser(1)*np.cos(ksi)+1j*eps_laser(2)*np.sin(ksi)


    #print "kp: %s"%(k_laser)
    #print "k: %s"%(k_photon)
    #print "pp: %s"%(p_pos)
    #print "p: %s"%(p_el)


    return (p_pos,p_el,k_laser,k_photon,q_pos,eps_m,eps_p)

In [4]:
p_pos,p_el,k_laser,k_photon,q_pos,eps_m,eps_p = kinematik()

In [5]:
V = np.array([qft.SpinorV((p_pos, m),s1) for s1 in [1,2]])
Ubar = np.array([qft.SpinorUBar((p_el,m),s1) for s1 in [1,2]])

print V.shape
print Ubar.shape

(2,)
(2,)


In [21]:
d_el  = m*a0/(4.*k_laser*p_el)
d_pos = m*a0/(4.*k_laser*p_pos)

fd=lambda x: qft.feyndagg(x)

In [22]:
J_00  = lambda l: fd(eps_photon(l))
#print "j0: %s"%(Ubar[r-1]*J_00(lp)*V[rp-1])
J_11  = lambda l: d_el * fd(eps_m)* fd(k_laser)   * fd(eps_photon(l)) - d_pos * fd(eps_photon(l))*fd(k_laser)* fd(eps_m)
J_1_1 = lambda l: d_el * fd(eps_p)* fd(k_laser)   * fd(eps_photon(l)) - d_pos * fd(eps_photon(l))*fd(k_laser)* fd(eps_p)
J_20  = lambda l: -4.  *  k_laser * eps_photon(l) * d_pos * d_el  * fd(k_laser)
J_22  = lambda l: 0.5  *  J_20(l) * (cos(ksi)**2  - sin(ksi)**2)
J_2_2 = lambda l:         J_22(l)

In [23]:
a = sigma/w_laser
t=J_00(1)
print t.shape

(4, 4)


In [9]:
def g(phi,sigma,Envelope):

    if Envelope == 'cos^2':
        return np.cos(np.pi*phi/2./sigma)**2*(phi>-sigma)*(phi<sigma)
    
    
            
def Int_g_2(phi,sigma,Envelope):
    
    if   Envelope == 'cos^2':
        a = np.pi/2./sigma
        Wert = (  0.25  * np.cos(a*phi)**3* np.sin(a*phi) \
                + 3./8. * np.cos(a*phi)   * np.sin(a*phi) \
                + 3./8. * (phi)  *a ) /a         * (phi>=-sigma)*(phi<=sigma)\
                + 3./8. *  sigma *(phi>sigma)

        return Wert 

In [10]:
def A_m_n(M,N,x_plus,p_el,p_pos,k_photon,k_laser):

    def f1(p):
        return -(m*a0)/(pk(p)) * g(phi,sigma,Envelope) *( pe(1,p) * np.cos(ksi) * np.cos(phi) + pe(2,p) * np.sin(ksi) * np.sin(phi) )
        
    def f2(p):
        return -(m*a0)**2/(2.*pk(p))*g(phi,sigma,Envelope)**2*((np.cos(ksi)*np.cos(phi))**2+(np.sin(ksi)*np.sin(phi))**2)          
    
    def f(p):
        return f1(p)+f2(p)
    
    def f1_SVEA(p):
        return -(m*a0)/(pk(p))*g(phi,sigma,Envelope)*(pe(1,p)*np.cos(ksi)*np.sin(phi)-pe(2,p)*np.sin(ksi)*np.cos(phi))

    def f2_SVEA(p):
        return -(m*a0)**2/(4.*pk(p))*(Int_g_2(phi,sigma,Envelope)+g(phi,sigma,Envelope)**2*np.cos(phi)*np.sin(phi)*(np.cos(ksi)**2-np.sin(ksi)**2))

    def f_SVEA(p):
        return f1_SVEA(p)+f2_SVEA(p)

    pk  =  lambda imp:    (imp * k_laser)
    pe  =  lambda l,imp:  (imp * eps_laser(l))
    
    P_  =  p_pos.minus() + p_el.minus() - k_photon.minus()    
    s   =  P_/k_laser.minus()

    phi =  w_laser       * x_plus
    
    H_plus =  s*phi - f_SVEA(p_el) + f_SVEA(-p_pos)

    if M == 0:        
        A       = -1./s *  (f(-p_pos) - f(p_el)) * exp(1j  * H_plus)
    
    else:
        A       =  g(phi,sigma,Envelope)**M *np.exp(  1j* ( H_plus + N*phi))
    
    return A  

def A_0_0  (A11,A1_1,A20,A22,A2_2):
    

    
    p_pos,p_el,k_laser,k_photon,q_pos,eps_m,eps_p = kinematik()
    
    pk  =  lambda p:    (p * k_laser)
    d_p = lambda p: m*a0 / ( 4.* pk(p) )    
        
    P_  =  p_pos.minus() + p_el.minus() - k_photon.minus()    
    s   =  P_/k_laser.minus()
        
    Wert = 2./s * (  ( d_p(p_pos)*p_pos*eps_m - d_p(p_el)*p_el*eps_m ) * A11  \
                   + ( d_p(p_pos)*p_pos*eps_p - d_p(p_el)*p_el*eps_p ) * A1_1 \
                   -  k_laser*k_photon*d_p(p_pos)*d_p(p_el)                   \
                   * ( 2.*A20 + (np.cos(ksi)**2 - np.sin(ksi)**2) * (A22 + A2_2) )  )
    
    return Wert


In [11]:
from sftrident.qft import asimps

def integrate(p_el,p_pos,k_photon,k_laser,M,N):
        return asimps.asimps( lambda x: A_m_n(M,N,x,p_el,p_pos,k_photon,k_laser),- a, a, Nx=421 , errorabs = 1e-4, maxrecur = 100 )

In [12]:
A_11  = integrate(p_el,p_pos,k_photon,k_laser,1,1)
A_1_1 = integrate(p_el,p_pos,k_photon,k_laser,1,-1)
A_20  = integrate(p_el,p_pos,k_photon,k_laser,2,0)
A_22  = integrate(p_el,p_pos,k_photon,k_laser,2,2)
A_2_2  = integrate(p_el,p_pos,k_photon,k_laser,2,-2)
A_00  = A_0_0(A_11[0],A_1_1[0],A_20[0],A_22[0],A_2_2[0] )


In [27]:
Lambda_1  = np.array([ J_00(l)*qft.parray(A_00) + ( J_11(l)*qft.parray(A_11[0])+J_1_1(l)*qft.parray(A_1_1[0])+\
                J_20(l)*qft.parray(A_20[0]) ) for l in [1,2]])

In [40]:
print type(Lambda_1[0])
print np.shape(Lambda_1[0])
print Lambda_1.shape
print Lambda_1[np.newaxis,:,np.newaxis].shape

<class 'sftrident.qft.spinors.DiracMatrix'>
(4, 4, 5, 5)
(2,)
(1, 2, 1)


In [34]:
print V[:,np.newaxis,np.newaxis].shape

(2, 1, 1)


In [35]:
test = Lambda_1[np.newaxis,:,np.newaxis]*V[:,np.newaxis,np.newaxis]

In [36]:
print test.shape

(2, 2, 1)
